<a href="https://colab.research.google.com/github/fliropp/colabs/blob/master/magnus_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Autentisiering**

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


**Last inn relevante pakker**

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

**Kjør colab_data_table-extension**


Colab includes an extension that renders pandas dataframes into interactive displays that can be filtered, sorted, and explored dynamically.

In [ ]:
%load_ext google.colab.data_table

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Run query and save as data frame
%%bigquery df_reading_vestby  --project amedia-analytics-eu
select
    user.user_key,
    main_content.id as content_id,
    round(sum(page.attentiontime)/1000, 1) as tot_attention_s
  from `amedia-adp-prod.facts3.pageviews` p
  left join `amedia-adp-marts.content_info.content_info_articles` a on p.main_content.id = a.content_id
  where _partitiondate >= current_date()-28
    and site_key = 'vestby'
    and page.flag.premium
    and page.attentiontime >= 20000
    and user.subscription_access
    and a.published_date >= current_date()-28
  group by 1,2
  having tot_attention_s >= 20

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_reading_vestby.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_key,content_id,tot_attention_s
0,d1584c1d-3e41-4693-b7a6-99579ab12aac,5-53-856222,37.3
1,90b79947-30dc-4719-a274-a3ad7b01659c,5-53-856222,34.3
2,33c6dc50-5f12-4e44-9f81-69fd4e932dc5,5-53-856222,149.4
3,fed61077-d383-47bd-81fe-61b98cea1673,5-53-858026,69.0
4,1dc0d978-a8f8-49ef-821a-105c09877733,5-53-858026,36.1


In [ ]:
baskets = df_reading_vestby[['user_key', 'content_id', 'tot_attention_s']]
baskets = baskets.copy()
baskets['tot_attention_s'] = baskets['tot_attention_s'].astype('int')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
baskets = baskets.groupby(['user_key', 'content_id'])['tot_attention_s'].sum().unstack().reset_index().fillna(0).set_index('user_key')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def one_hot_encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

baskets = baskets.applymap(one_hot_encode)
baskets.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


content_id,5-53-849584,5-53-851132,5-53-851363,5-53-853147,5-53-853205,5-53-853232,5-53-853253,5-53-853327,5-53-853336,5-53-853408,...,5-53-861691,5-53-861700,5-53-861760,5-53-861819,5-53-861836,5-53-861880,5-53-861889,5-53-861909,5-53-861942,5-53-861977
user_key,,,,,,,,,,,,,,,,,,,,,
00082195-42cb-4008-b4cd-9ce0598ce258,1,1,0,1,0,0,0,1,0,1,...,0,1,1,1,1,0,0,0,0,1
001564ef-a82d-49a8-9829-328c9debfdaa,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
00456f25-eb2b-4e9f-8a64-4551e10cb0b9,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
00477dd9-2582-4d6e-a0d4-3fbc06f08399,1,0,1,1,0,1,0,0,0,0,...,0,1,0,1,0,1,0,1,1,1
0059c84d-d621-4d75-9c9d-99cd04967342,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
itemsets = apriori(baskets,
                           use_colnames=True,
                           verbose=1,
                           low_memory=False,
                           min_support=0.1
                          )

itemsets.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 30 combinations | Sampling itemset size 6


,support,itemsets
0,0.203023,(5-53-849584)
1,0.199316,(5-53-851363)
2,0.220701,(5-53-853147)
3,0.117479,(5-53-853205)
4,0.309666,(5-53-853327)


In [ ]:
itemsets['length'] = itemsets['itemsets'].apply(lambda x: len(x))
itemsets.head(4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
0,0.203023,(5-53-849584),1
1,0.199316,(5-53-851363),1
2,0.220701,(5-53-853147),1
3,0.117479,(5-53-853205),1


In [ ]:
itemsets['length'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3    2507
2    2226
4     415
1     170
5       7
Name: length, dtype: int64

In [ ]:
rules = association_rules(itemsets, metric="lift", min_threshold=0.5)
rules = rules[(rules['confidence'] > 0.7)]
rules.head(20).sort_values(by='lift', ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
710,(5-53-859643),(5-53-854022),0.164243,0.373824,0.122042,0.743056,1.987716,0.060644,2.437010,0.594563
754,(5-53-860952),(5-53-854022),0.199601,0.373824,0.140006,0.701429,1.876362,0.065390,2.097241,0.583526
262,(5-53-853408),(5-53-855650),0.146849,0.444539,0.122327,0.833010,1.873871,0.057047,3.326304,0.546615
275,(5-53-853412),(5-53-855650),0.166239,0.444539,0.124038,0.746141,1.678458,0.050138,2.188064,0.484809
977,(5-53-854129),(5-53-855650),0.156259,0.444539,0.115483,0.739051,1.662509,0.046020,2.128618,0.472301
404,(5-53-853734),(5-53-855650),0.246650,0.444539,0.181066,0.734104,1.651381,0.071421,2.089015,0.523589
39,(5-53-851363),(5-53-855650),0.199316,0.444539,0.140861,0.706724,1.589789,0.052257,1.893985,0.463336
898,(5-53-854057),(5-53-855650),0.219561,0.444539,0.154548,0.703896,1.583428,0.056945,1.875897,0.472117
285,(5-53-853412),(5-53-860154),0.166239,0.544055,0.130596,0.785592,1.443957,0.040153,2.126529,0.368761
570,(5-53-854001),(5-53-860154),0.235244,0.544055,0.181637,0.772121,1.419198,0.053651,2.000824,0.386236
